This notebook looks at the Lorenz Model III as described in Lorenz, 2005.
This model has a long and short scale that interact with eachother.
The state variable is $Z$ and contains both scales.
$Z$ is smoothed using a window defined by $I$ and weights defined by $\alpha$ and $\beta$, generating the variable $X$ which contains the long scales.
The variable $Y$ contains the short scales and is equal to $Z - X$.

The time derivative of $Z$ is very simmilar to the Lorenz-96 model:
$$
d Z_n/dt = [X, X]_{K, n} + b^2 [Y, Y]_{1, n} + c[Y, X]_{1, n} - X_n - bY_n + F
$$
$$
X_n = \sum_{i = -I}^I (\alpha - \beta |i|) Z_{n + i}
$$
Where the above has the first and last terms divided by 2.
$$
Y_n = Z_n - X_n
$$
$$
[X, Y]_{K, n} = \sum_{j=-J}^{J}\sum_{i=-J}^{J}( -X_{n - 2K - i}Y_{n-K-j} + X_{n - K + j - i}Y_{n + K + j})/K^2
$$
For an odd $K$, $J = (K-1)/2$.
For an even $K$, $J=K/2$ and the above two sums have the first and last terms divided by 2.

In order to make this model work with a coarse model (too coarse to capture the behavior of $Y$) I estimate the terms involving $Y$ using the fine ensemble.
Therefore, the fine ensemble effects the coarse ensemble (each fine ensemble member effects multiple coarse ensemble members), but the coarse ensemble does not effect the fine ensemble.

In [1]:
import importlib
import time
import os
import pickle
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy import integrate
from scipy import interpolate

In [2]:
from multi_loc import covariance, assimilate, utilities

In [3]:
%matplotlib inline

In [4]:
def imshow(X, title=None, vmin=None, vmax=None, cmap=None):
    plt.figure()
    im = plt.imshow(X, vmin=vmin, vmax=vmax, cmap=cmap)
    plt.colorbar(im)
    plt.title(title)

In [6]:
load_files = True

In [12]:
N_Z = 960
K = 32
I = 12
F = float(15)
b = float(10)
# b = float(7)
c = float(2.5)
alpha = (3 * I**2 + 3) / (2 * I**3 + 4 * I)
beta = (2 * I**2 + 1) / (I**4 + 2 * I**2)

coarse = 8
N_Xc = N_Z//coarse
K_c = K//coarse

N_eZ = 20
N_eX = 500
save_dir = 'fine_time'
home = os.path.expanduser('~')
save_dir = os.path.join(
    home,
    'Documents/python_code/multi_loc/notebooks/',
    save_dir)

dt_gt = 0.005/8

dt_kf = 0.005
gt_cycles = 5000
T_gt = dt_kf * gt_cycles

t_gt = np.linspace(0, T_gt, int(T_gt/dt_gt) + 1)
t_lr = np.linspace(0, T_lr, int(T_lr/dt_lr) + 1)

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

this_path = os.path.join(save_dir, 't_gt')
np.save(this_path, t_gt)

In [16]:
run_vars = {'Nz': N_Z,
            'K': K,
            'I': I,
            'F': F,
            'b': b,
            'c': c,
            'alpha': alpha,
            'beta': beta,
            'coarse': coarse,
            'Nez': N_eZ,
            'Nex': N_eX,
            'dt_gt': dt_gt,
            'Tgt': T_gt
            }
this_path = os.path.join(save_dir, 'run_vars.pkl')
with open(this_path, mode='wb') as f:
    pickle.dump(run_vars, f)

# Generate Truth run

In [29]:
X_interp = np.random.uniform(low=-5, high=10, size=N_Z//K)
X_interp = np.concatenate([X_interp, [X_interp[0]]])
x = np.arange(N_Z)
x_interp = x[::K]
x_interp = np.concatenate([x_interp, [x[-1] + 1]])
f_X = interpolate.interp1d(x_interp, X_interp, kind='quadratic')
X0 = f_X(x)

Y0 = np.random.uniform(low=-0.5, high=1, size=N_Z)

Z0 = X0 + X0*Y0/X0.max()

# t = np.linspace(0, 20, 100)

Z = utilities.return_LM3_data(
    Z0, dt_rk, T_ic, dt_rk, K=K, I=I, F=F, b=b, c=c)

Z0 = Z[:, -1]

In [17]:
this_path = os.path.join(
    home,
    'Documents/python_code/multi_loc/notebooks/RK_tune',
    'Z_GT.npy'
)
Z0 = np.load(this_path)[:, 0]

In [21]:
Z_GT = utilities.return_LM3_data(
    Z0, dt_gt, T_gt, dt_gt, K=K, I=I, F=F, b=b, c=c)

this_path = os.path.join(save_dir, 'Z_GT')
np.save(this_path, Z_GT)
# Z_GT = np.load('./Z_GT.npy')